In [ ]:
import pandapower as pp
import pandapower.networks
import pandas as pd
import plotly.express as px
import numpy as np
import tensorflow as tf
import os
import time

import matplotlib.pyplot as plt
%matplotlib inline
from pandapower.plotting.plotly import simple_plotly
from pandapower.plotting.plotly import vlevel_plotly
from pandapower.plotting.plotly import pf_res_plotly

from pandapower.timeseries import DFData
from pandapower.timeseries import OutputWriter
from pandapower.timeseries.run_time_series import run_timeseries
from pandapower.control import ConstControl

In [ ]:
net = pp.networks.mv_oberrhein()
net1, net2 = pp.networks.mv_oberrhein(separation_by_sub=True)

In [ ]:
net = net1

In [ ]:
# a = [int(i.split(' ')[-1]) for i in net.bus['name']]
bus_indices = net.bus.index

c = {}
l = 0 
s = 0
for i in bus_indices:
	c[i] = [0,0]
	if(i in list(net.load['bus'])):
		c[i][0]+=1
		l+=1
	if(i in list(net.sgen['bus'])):
		c[i][1]+=1
		s+=1
print('{bus index, [load,gen]}')
print(c)

l = [[],[],[],[]]
for k,v in c.items():
	if(v[0]==0 and v[1]==0): #nothing
		l[0].append(k) 
	if(v[0]>0 and v[1]==0): #only load
		l[1].append(k)
	if(v[0]==0 and v[1]>0): #only sgen
		l[2].append(k)
	if(v[0]>0 and v[1]>0): #both
		l[3].append(k)
print('\nNothing: ', l[0])
print('Only load: ', l[1])
print('Only sgen: ', l[2])
print('Both: ', l[3])
print(f'Ratio: Nothing/all: {( len(l[0])/( len(l[0])+len(l[1])+len(l[2])+len(l[3]) ) ):.3f}')

In [ ]:
# a = [int(i.split(' ')[-1]) for i in net.bus['name']]
bus_indices = net.bus.index

c = []
nc = []
l = 0
for i in net.load.to_numpy():
	if(i[1] in bus_indices):
		c.append(i[1])
		l+=1
	else:
		nc.append(i[1])
print(f'Loads connected to something: {c}')
print(f'Loads connected to nothing: {nc}')

print(f'\nLoads-> total: {len(net.load)}, connected to some bus: {l} (ratio: {(l/len(net.load)):.3f})')

In [ ]:
# a = [int(i.split(' ')[-1]) for i in net.bus['name']]
bus_indices = net.bus.index

c = []
nc = []
l = 0
for i in net.sgen.to_numpy():
	if(i[1] in bus_indices):
		c.append(i[1])
		l+=1
	else:
		nc.append(i[1])
print(f'Sgens connected to something: {c}')
print(f'Sgens connected to nothing: {nc}')

print(f'\nSgens-> total: {len(net.sgen)}, connected to some bus: {l} (ratio: {(l/len(net.sgen)):.3f})')

In [ ]:
print(net.load[net.load['scaling'] == 0].count())
print(net.sgen[net.sgen['scaling'] == 0].count()) #All sgen scaling value set to 0!!!! I was wandering why changing sgens time series didn't bring to different resutls -.-

In [ ]:
#All the scaling values are 0.6
net.load['scaling'] = 0.6
net.sgen['scaling'] = 0.6
#Can be changed also in the Controllers

In [ ]:
def num_foreach_element(arr):
	d = {}
	for i in arr.to_numpy():
		i = i[0]
		d[i] = d[i]+1 if i in d.keys() else 1
	return d

In [ ]:
input_dir = './TimeSeries/1-MVLV-rural-all-0-sw/'
n_timesteps = 4 * 24 * 365

#Loads dataset
profile_load = pd.DataFrame()
n_load = len(net.load)
n_res = len(net.sgen)
# The parameter “sR” generally describes the nominal apparent power of power plants, distributed energy resources and loads
loads = pd.read_csv(f'{input_dir}Load.csv', sep=';')
loads_timeseries = pd.read_csv(f'{input_dir}LoadProfile.csv', sep=';')

RESs = pd.read_csv(f'{input_dir}RES.csv', sep=';')
RESs_timeseries = pd.read_csv(f'{input_dir}RESProfile.csv', sep=';')

#Cases:
# 0: low load, high generation
# 1: normal load and generation
# 2: high load, low generation
case = 2
if(case==0):
	scale_factor_load = 0.2
	scale_factor_sgen = 2
elif(case==1):
	scale_factor_load = 1
	scale_factor_sgen = 1
elif(case==2):
	scale_factor_load = 0.8/0.6
	scale_factor_sgen = 0.2
print(f'Case: {case}, scale load: {scale_factor_load}, scale sgen: {scale_factor_sgen}')
'''
# Papers:
# -SimBench—A Benchmark Dataset of Electric Power Systems to Compare Innovative Solutions Based on Power Flow Analysis
# -https://publica.fraunhofer.de/eprints/urn_nbn_de_0011-n-5554297.pdf
Germany standard load profiles(SLPs): Commercial enterprises (G), households (H), agricultural holdings (L) and industrial companies
(BL/BW) were considered as accumulated consumers, while the provided time series for electric
vehicles (EVs) and heat pumps (HPs) were interpreted as individual consumers.
Ending letter: A-C low consumption, M medium, H high
#print(set(loads_timeseries.columns)) 
'''
loads_type = loads.loc[:n_load-1,['profile']] #loc ranges is [0,n], not [0,n[. Find a better/more uniform way to select the loads
temp_profile_p = []
temp_profile_q = []
print(f'Load elements by type: {num_foreach_element(loads_type)}')
for l in loads_type.to_numpy():
	temp_profile_p.append(loads_timeseries[f'{l[0]}_pload'])
	temp_profile_q.append(loads_timeseries[f'{l[0]}_qload'])

#Loads p (in MW) 
profile_load_p = pd.concat(temp_profile_p,axis=1)[:n_timesteps]
profile_load_p.columns = net.load.index

ds_lp = DFData(profile_load_p)
cc_lp = ConstControl(net, 'load', 'p_mw', element_index=net.load.index, profile_name=profile_load_p.columns,
					data_source=ds_lp, scale_factor=scale_factor_load)

#Loads q (in MVar)
profile_load_q = pd.concat(temp_profile_q,axis=1)[:n_timesteps]
profile_load_q.columns = net.load.index
ds_lq = DFData(profile_load_q)
cc_lq = ConstControl(net, 'load', 'q_mvar', element_index=net.load.index, profile_name=profile_load_q.columns,
					data_source=ds_lq, scale_factor=scale_factor_load)

#RES p (in MW)
res_to_add = pd.DataFrame(['WP4','WP7','WP4','WP7','WP4','WP7'], columns=['profile'])
RESs_type = RESs.loc[:n_res-1-len(res_to_add),['profile']]
RESs_type = pd.concat([RESs_type, res_to_add])
temp_profile_p = []
print(f'RES elements by type: {num_foreach_element(RESs_type)}')
for l in RESs_type.to_numpy():
	temp_profile_p.append(RESs_timeseries[f'{l[0]}'])
	# temp_profile_p.append(RESs_timeseries[f'{l[0]}_pload']) #Q values are not required

profile_res_p = pd.concat(temp_profile_p,axis=1)[:n_timesteps]
profile_res_p.columns = net.sgen.index
ds_sp = DFData(profile_res_p)
cc_sp = ConstControl(net, 'sgen', 'p_mw', element_index=net.sgen.index, profile_name=profile_res_p.columns,
					data_source=ds_sp, scale_factor=scale_factor_sgen)

In [ ]:
figsize=(20, 3)
fig = plt.figure(figsize=figsize)
(cc_lp.data_source.df*cc_lp.scale_factor).sum(axis=1).plot(label='load p',ylabel='mw')
# (cc_lq.data_source.df*cc_lq.scale_factor).sum(axis=1).plot(label='load q',ylabel='mw')
(cc_sp.data_source.df*cc_sp.scale_factor).sum(axis=1).plot(label='sgen')
# profile_res_p.loc[:,['PV5', 'PV8', 'PV6']].sum(axis=1).plot(label='sgen PV')
# profile_res_p.loc[:,['WP4','WP7']].sum(axis=1).plot(label='sgen WP')
plt.legend()
plt.show()

In [ ]:
df = profile_res_p
df =  df.loc[:,~df.columns.duplicated()]
df.boxplot(rot=45,figsize=figsize)
plt.ylabel('mw (?)')

In [ ]:
time_steps = range(0,n_timesteps)

output_dir = os.path.join(input_dir,'Results')
ow = OutputWriter(net, time_steps, output_path=output_dir, output_file_type=".xlsx", log_variables=list())

#Save time series (output)
# these variables are saved to the harddisk after / during the time series loop
ow.log_variable('res_load', 'p_mw')
ow.log_variable('res_bus', 'vm_pu')
ow.log_variable('res_line', 'loading_percent')
ow.log_variable('res_line', 'i_ka')
#Add net.res_trafo['loading_percent']

print('Time steps: ',len(time_steps), '. Num Loads: ', net.load.index.shape, '. Load p and q: ', profile_load_p.shape, profile_load_q.shape, '. Num RESs: ', net.sgen.index.shape, '. RESs p: ',profile_res_p.shape)
t1 = time.time()
run_timeseries(net,time_steps)
t2 = time.time()
print(f'Simulation time : {(t2-t1):.2f} s')

#Save time series (input)
path = os.path.join(output_dir, "loads_p.xlsx")
profile_load_p.to_excel(path)
path = os.path.join(output_dir, "loads_q.xlsx")
profile_load_q.to_excel(path)
path = os.path.join(output_dir, "RESs_p.xlsx")
profile_res_p.to_excel(path)
t3 = time.time()
print(f'Saving files time: {(t3-t2):.2f} s')

In [ ]:
def plot_df(df,title='',y_axis='',file_name=''):
	fig = px.line(df,x=time_steps, y=df.columns, width=800, height=400)
	fig.update_layout(title=title,
					xaxis_title='time steps',
					yaxis_title=y_axis)
	if(file_name):
		fig.write_html(file_name)
		print(f'Saved {title} in {file_name}')

# voltage results
vm_pu_file = os.path.join(output_dir, "res_bus", "vm_pu.xlsx")
vm_pu = pd.read_excel(vm_pu_file, index_col=0)
plot_df(vm_pu,'buses voltage magnitude', 'bus vm [pu]', os.path.join(output_dir, "Plots", "bus vm.html"))

# # line loading results
ll_file = os.path.join(output_dir, "res_line", "loading_percent.xlsx")
line_loading = pd.read_excel(ll_file, index_col=0)
plot_df(line_loading,'line_loading', 'line_loading [%]', os.path.join(output_dir, "Plots", "line load.html"))

# # load results
load_file = os.path.join(output_dir, "res_load", "p_mw.xlsx")
load = pd.read_excel(load_file, index_col=0)
plot_df(load,'load active power', 'p [MW]', os.path.join(output_dir, "Plots", "load.html"))

In [ ]:
#Plot network and loadings

bus = 58
fig = simple_plotly(net, bus_size=5, ext_grid_size=10)
fig.add_trace(px.scatter(x=[net.bus_geodata.loc[bus, 'x']], y=[net.bus_geodata.loc[bus, 'y']],color=['r'],size=[10]).data[0])
# _ = vlevel_plotly(net, bus_size=5, ext_grid_size=10)
# fig = pf_res_plotly(net, bus_size=8)
# fig.write_html(f"images/MVOberrhein/Half2.html")
# fig.write_image(f"images/MVOberrhein/Half2.png")

In [ ]:
print(vm_pu[58]) #constant voltage?
print(net.bus.loc[95])
print(net.load[net.load['bus']==58]) #no load in bus 58

In [ ]:
# vm_pu.plot(label="vm_pu", figsize=figsize)